## Instalações (RODAR APENAS UMA VEZ ESSA PARTE):

In [ ]:
#!mv * python

!zip -r /content/file.zip /content/python

In [ ]:
!pip install openai --target .

In [ ]:
!pip install openai
!pip install pymysql
!pip install mysql-connector

## Pré-processamento:

In [ ]:
import pandas as pd

import pymysql
import sqlalchemy

import openai

In [ ]:
api_key = '###'

In [ ]:
database_username = '###'
database_password = '###'
database_ip       = '###'
database_name     = '###'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                            format(database_username, database_password,
                                                    database_ip, database_name))

In [ ]:
prompt = """
### propriedades das tabelas em mysql:
#
# 'CREATE TABLE `pedidos` (\n  `cnpj` bigint DEFAULT NULL,\n  `centroDeCusto` text,\n  `codFranqueador` int DEFAULT NULL,\n  `codLoja` int DEFAULT NULL,\n  `datMovimento` text,\n  `dataVenda` text,\n  `controle` int DEFAULT NULL,\n  `controlePdv` int DEFAULT NULL,\n  `tipoVenda` int DEFAULT NULL,\n  `numPdv` int DEFAULT NULL,\n  `numAbertura` int DEFAULT NULL,\n  `cancelada` text,\n  `valTroco` text,\n  `valDesconto_x` double DEFAULT NULL,\n  `valDescontoItem` double DEFAULT NULL,\n  `valAcrescimo` double DEFAULT NULL,\n  `valProduto` double DEFAULT NULL,\n  `valLiquido` double DEFAULT NULL,\n  `docConsumidor` bigint DEFAULT NULL,\n  `docConsumidorDesconto` text,\n  `possuiCpfCnpj` text,\n  `mesa` json DEFAULT NULL,\n  `cartao` json DEFAULT NULL,\n  `numLancto` int DEFAULT NULL,\n  `codProduto` int DEFAULT NULL,\n  `descricaoProduto` text,\n  `codBarras` text,\n  `unidadeProduto` text,\n  `quantidade` int DEFAULT NULL,\n  `valUnitario` text,\n  `valTotal` text,\n  `valDesconto_y` int DEFAULT NULL,\n  `datHoraLancamento` text,\n  `cancelado` text,\n  `numLanctoItemPrincipal` int DEFAULT NULL,\n  `posicaoClienteMesa` int DEFAULT NULL,\n  `composicaoCodigo` int DEFAULT NULL,\n  `codGrupoCombinacao` int DEFAULT NULL,\n  `nomUsuarioVenda` text,\n  `nomUsuarioCancelamento` text,\n  `momentoCancelamento` text,\n  `ordemSefaz` int DEFAULT NULL,\n  `enviadoParaSefaz` text,\n  `reshop` text,\n  `composicao.tipo` text,\n  `composicao.descricao` text,\n  `fiscal.codCfop` int DEFAULT NULL,\n  `fiscal.cstCofins` text,\n  `fiscal.codOrigemCst` int DEFAULT NULL,\n  `fiscal.codCstIcms` text,\n  `fiscal.valorDescontoItem` int DEFAULT NULL,\n  `fiscal.codModalidadeIcms` int DEFAULT NULL,\n  `fiscal.codCstIpi` text,\n  `fiscal.codCstPis` text,\n  `fiscal.codCsosn` int DEFAULT NULL,\n  `fiscal.codModalidadeIcmsSubs` int DEFAULT NULL,\n  `fiscal.perIcms` int DEFAULT NULL,\n  `fiscal.perReducaoBaseCalculoIcms` int DEFAULT NULL,\n  `fiscal.perIcmsSubs` int DEFAULT NULL,\n  `fiscal.perReducaoBaseCalculoIcmsSubs` int DEFAULT NULL,\n  `fiscal.perPis` int DEFAULT NULL,\n  `fiscal.perCofins` int DEFAULT NULL,\n  `fiscal.perIpi` int DEFAULT NULL,\n  `fiscal.perSimplesNacional` int DEFAULT NULL,\n  `fiscal.valBaseCalculoIcms` text,\n  `fiscal.valBaseCalculoIcmsSubs` int DEFAULT NULL,\n  `fiscal.valBaseCalculoIcmsRetidoFonte` int DEFAULT NULL,\n  `fiscal.valBaseCalculoPis` text,\n  `fiscal.valBaseCalculoCofins` text,\n  `fiscal.valBaseCalculoIpi` text,\n  `fiscal.valIcms` int DEFAULT NULL,\n  `fiscal.valIcmsSubs` int DEFAULT NULL,\n  `fiscal.valIcmsRetidoFonte` int DEFAULT NULL,\n  `fiscal.valIcmsSimplesNacional` int DEFAULT NULL,\n  `fiscal.valPis` int DEFAULT NULL,\n  `fiscal.valCofins` int DEFAULT NULL,\n  `fiscal.valIpi` int DEFAULT NULL,\n  `fiscal.valFrete` int DEFAULT NULL,\n  `fiscal.valSeguro` int DEFAULT NULL,\n  `fiscal.valOutrasDespesas` int DEFAULT NULL,\n  `fiscal.codItemEstoque` int DEFAULT NULL,\n  `fiscal.ncm` text,\n  `fiscal.cest` int DEFAULT NULL,\n  `numPedido` int DEFAULT NULL,\n  `codCliente` int DEFAULT NULL,\n  `codPrePedido` text,\n  `horAbertura` text,\n  `horEnvioProducao` text,\n  `horExpedicao` text,\n  `horFechamento` text,\n  `valTaxaEntrega` double DEFAULT NULL,\n  `valTaxaEntregaAdicional` double DEFAULT NULL,\n  `valTaxaEntregaAlterada` text,\n  `importadoCallCenter` text,\n  `horAgendamento` text,\n  `justificativaCancelamento` text,\n  `observacao` text,\n  `sequencia` int DEFAULT NULL,\n  `codFormaPagamento` int DEFAULT NULL,\n  `valor` double DEFAULT NULL,\n  `tefSequencia` int DEFAULT NULL,\n  `tefData` text,\n  `tefNsu` int DEFAULT NULL,\n  `tefAutorizacao` text,\n  `tefQtdeParcela` int DEFAULT NULL,\n  `receita` text,\n  `bandeira` text,\n  `formaPgto` text,\n  `codigoBandeiraNfce` int DEFAULT NULL,\n  `codigoFinalizadorNfce` int DEFAULT NULL,\n  KEY `idx_relatorio_vendas_cancelada_cancelado` (`cancelada`(10),`cancelado`(10))\n) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci'
# Use a view chamada pedidos para fazer as queries relacionadas a pedidos.
# Coluna chamada controle é o identificador único de cada pedido, então para contabilizar os pedidos, considere isso para não contar o mesmo pedido mais de uma vez.
# a coluna 'quantidade' é o número de unidades que foi vendido daquele determinado produto.

### faça uma query para responder à seguinte pergunta:
# qual foi o dia com mais pedidos na loja 1 em março de 2023?

# Coluna de data chama datMovimento
# Não se esqueça que a coluna 'controle' é um id único de cada pedido, então cuidado para não contar mais de uma vez um pedido (dica: use o distinct para tratar isso).
# Hoje é dia 04/04/2023.
# Estamos no ano de 2023.
SELECT
"""

In [ ]:
# APRENDER UM BANCO DE DADOS NOVO PEGANDO OS NOMES DAS TABELAS E COLUNAS
# CRIAR ASSUNTOS DAS PERGUNTAS PARA DIRECIONAR QUAIS SCHEMAS PEGAR NO BANCO DE DADOS
# FAZER ELE MANTER O CONTEXTO ENTRE PERGUNTAS PARA APRENDER COM O QUE O USUARIO PERGUNTA

# INCLUIR ETAPA DE RECEBER UM AUDIO, CONVERTER PARA TEXTO E PASSAR PARA A API
# IMPEDIR QUE FACA QUERIES DE UPDATE, INSERT E DELETE (USUARIO DO SISTEMA SOH PODE TER PERMISSAO DE LEITURA)
# PENSAR EM DAR DIFERENTES RETORNOS

# BOLAR UMA CARA PARA O APP OU SISTEMA WEB

In [ ]:
# NAO DEU CERTO!!!!
#prompt = """
## e na loja de 'Pinheiros'?
#SELECT
#"""

In [ ]:
# faça uma query para calcular o total de pedidos da loja 1 no mês de fevereiro
# faça uma query para descobrir qual a loja que teve mais pedidos em dezembro do ano 2022.
# faça uma query para descobrir qual é o produto mais vendido em cada loja no ano de 2023?
# Retorne o nome da loja (não precisa do código da loja), apenas o nome do produto mais vendido e a quantidade de unidades vendidas desse produto. Liste todas as lojas.
# qual produto é mais vendido às 12h?

# quantas unidades do produto 'CHOCOLOVER' foram vendidos na loja 'Pinheiros' no mês de janeiro de 2023?
# quantas unidades do produto 'SIMPSONS' foram vendidas nos últimos 30 dias?

# NAO DEU CERTO!!
# qual o dia de maior venda por loja?

In [ ]:
openai.api_key = api_key

response = openai.Completion.create(
  model="text-davinci-003",  # mais rapido: code-cushman-001
  prompt=prompt,
  temperature=0,
  max_tokens=300,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=["#", ";"]
)

In [ ]:
response

<OpenAIObject text_completion id=cmpl-71iMyM1aAqdWrlFuzfbMyeeN2dVV3 at 0x7f6c6c371360> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "  DATE_FORMAT(datMovimento, '%d/%m/%Y') AS data,\n  COUNT(DISTINCT controle) AS pedidos\nFROM pedidos\nWHERE YEAR(datMovimento) = 2023\n  AND MONTH(datMovimento) = 3\n  AND codLoja = 1\nGROUP BY data\nORDER BY pedidos DESC\nLIMIT 1"
    }
  ],
  "created": 1680643240,
  "id": "cmpl-71iMyM1aAqdWrlFuzfbMyeeN2dVV3",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 101,
    "prompt_tokens": 1902,
    "total_tokens": 2003
  }
}

In [ ]:
query = f'SELECT\n{response["choices"][0]["text"]}'
print(query)

SELECT
  DATE_FORMAT(datMovimento, '%d/%m/%Y') AS data,
  COUNT(DISTINCT controle) AS pedidos
FROM pedidos
WHERE YEAR(datMovimento) = 2023
  AND MONTH(datMovimento) = 3
  AND codLoja = 1
GROUP BY data
ORDER BY pedidos DESC
LIMIT 1


In [ ]:
pd.read_sql(query, con=database_connection)

,data,pedidos
0,08/03/2023,55
